In [ ]:
#default_exp wrangling

# Data wrangling
> Data wrangling related functionality

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export
from pathlib import Path
import pandas as pd

In [ ]:
#export
import re

_pattern = re.compile('_(D\d)')
def parse_distortion_severity(video_name):
    distortions = _pattern.findall(video_name)
    if len(distortions)==0: # reference video
        return 'D0', 'S0'
    sev_level = int(video_name[-1])
    assert sev_level in {1,2,3,4,5}
    return '_'.join(distortions), f"S{sev_level}"

test_eq(('D0', 'S0'), parse_distortion_severity('Airport'))
test_eq(('D9', 'S1'), parse_distortion_severity('Airport_D9_1'))
test_eq(('D3_D5', 'S1'), parse_distortion_severity('Airport_D3_2_D5_1'))
test_eq(('D2_D7_D9', 'S4'), parse_distortion_severity('Airport_D2_D7_D9_4'))

In [ ]:
#export

def parse_scene(video_name):
    return video_name.split("_D", 1)[0].lower()

test_eq('airport', parse_scene('Airport'))
test_eq('airport', parse_scene('Airport_D2_D7_D9_4'))
test_eq('airport', parse_scene('Airport_D2_3_D9_4'))

In [ ]:
#export
def label_dataframe(df):
    df['scene'] = df['video_name'].apply(parse_scene)
    df['raw_label'] = df['video_name'].apply(parse_distortion_severity)
    df['distortion'] = df['raw_label'].apply(lambda t: t[0])
    df['severity'] = df['raw_label'].apply(lambda t: t[1])
    return df

In [ ]:
#export
from sklearn.model_selection import train_test_split

def make_dataframe_splitter(valid_pct, strata='raw_label'):
    def stratified_split(df):
        _, val_index = train_test_split(df.index, test_size=valid_pct, stratify=df[strata])
        df['is_valid'] = False
        df.loc[val_index, 'is_valid'] = True
        return df
    return stratified_split

In [ ]:
#export
def populate_frames(df, frame_indices_list):
    for frame_indices in frame_indices_list:
        df = df.copy()
        df['frames'] = [frame_indices]*len(df)
        df['frame_paths'] = df['video_path'].apply(lambda vn: [str(Path(vn) / f"{i}.jpg") for i in frame_indices])
        yield df

def make_framer(frame_indices_list):
    return lambda dataf: pd.concat(list(populate_frames(dataf, frame_indices_list)), axis=0)

In [ ]:
#export

def remove_corrupt_video_frames(df):
    video_names = [
     'Concorde_place_D1_D5_1',
     'Concorde_place_D6_1',
     'Concorde_place_D1_D5_4',
     'Concorde_place_D1_D5_3',
     'Concorde_place_D1_D5_2',
     'Concorde_place_D6_2',
     'Concorde_place_D6_3',
     'Concorde_place_D6_4'
    ]
    tbd_idx = tbd_idx = df['video_name'].isin(video_names) & df['frames'].apply(lambda seq: max(seq) > 33)
    return df[~tbd_idx].copy()

In [ ]:
#export

def make_dataframe(root):
    df = pd.DataFrame(data=dict(video_path=[str(p) for p in sorted(root.ls())]))
    df['video_name'] = df['video_path'].apply(lambda p: Path(p).name)
    return df

In [ ]:
#export

def make_train_dataframe(root, valid_pct, frame_indices_list):
    return (
        make_dataframe(root)
        .pipe(label_dataframe)
        .pipe(make_df_splitter(valid_pct))
        .pipe(make_framer(frame_indices_list))
        .pipe(remove_corrupt_video_frames)
    )


In [ ]:
#export

def assert_stratied_split(df, label_col):
    train_df, val_df = df[~df['is_valid']], df[df['is_valid']]
    ratio = len(val_df) / len(train_df)
    label_freqs = pd.concat([train_df[label_col].value_counts(), val_df[label_col].value_counts() ], axis=1, join='outer')
    label_freqs.columns=['train', 'val']
    label_freqs['ratio'] = (label_freqs['val'] / label_freqs['train']).apply(abs)
    # assert that difference in class counts is less than 1% of dataset size
    assert ratio - 0.02 < label_freqs['ratio'].mean() < ratio + 0.02, label_freqs['ratio'].min()